# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not including data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

nhl_df1=nhl_df[nhl_df['year']==2018]
nhl_df1['WL']=nhl_df1['team']
nhl_df1['Full_Name']=nhl_df['team']
nhl_df1['Second_Name']=nhl_df['team']
nhl_df2=nhl_df1.set_index('team')
def win_loss(team):
    try:
        ratio=int(nhl_df2.loc[team,'W'])/(int(nhl_df2.loc[team,'L'])+int(nhl_df2.loc[team,'W']))
        result = ratio
    except:
        result= 'N/A'
    return result
nhl_df2['WL']=nhl_df2['WL'].apply(win_loss)
def sec(name):
    name1=name.split()[-1].strip()
    name2=name1.replace('*','')
    return name2
nhl_df2['Second_Name']=nhl_df2['Second_Name'].apply(sec)
nhl_df3=nhl_df2[['WL','Full_Name','Second_Name']]

def new(on):
    if on in New_Name:
        res=New_Name[on]
    else:
        res=on
    return res

New_Name={'Islanders':'RangersIslandersDevils'
            ,'Rangers':'RangersIslandersDevils'
            ,'Devils':'RangersIslandersDevils'
            ,'Jackets':'Blue Jackets'
            ,'Leafs':'Maple Leafs'
            ,'Knights':'Golden Knights'
            ,'Kings':'KingsDucks'
            ,'Wings':'Red Wings'
            ,'Ducks':'KingsDucks'}
nhl_df3['New_Map']=nhl_df3['Second_Name'].apply(new)
nhl_df4=nhl_df3[nhl_df3['WL']=='N/A']
nhl_df5=nhl_df3[~nhl_df3.index.isin(nhl_df4.index)]

def sec2(new):
    new=new.split('[')[0].strip()
    new=new.replace('—','')
    return new
cities['NHL']=cities['NHL'].apply(sec2)

merged=cities.merge(nhl_df5,left_on='NHL' ,right_on='New_Map')
merged2=merged[['Metropolitan area','WL','Population (2016 est.)[8]']]
merged2['Population (2016 est.)[8]']= merged2['Population (2016 est.)[8]'].apply(lambda x: float(x))
merged2['WL']= merged2['WL'].apply(lambda x: float(x))
merged3=merged2.groupby('Metropolitan area').agg({'WL':np.nanmean,'Population (2016 est.)[8]':np.nanmean})

In [2]:
def nhl_correlation(): 
    # YOUR CODE HERE

#    raise NotImplementedError()
    
    population_by_region =merged3['Population (2016 est.)[8]']
    # pass in metropolitan area population from cities
    win_loss_by_region = merged3['WL'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

#    assert len(population_by_region)==len(win_loss_by_region), "Q1: Your lists must be the same length"
#    assert len(population_by_region)==30, "Q1: There should be 30 teams being analysed for NHL"
    
    return float(stats.pearsonr(population_by_region, win_loss_by_region)[0])

In [3]:
type(nhl_correlation())

float

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [4]:
nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

nba_df=nba_df[nba_df['year']==2018]
nba_df['Full_Name']=nba_df['team']
nba_df
dic2={
    'Blazers':'Trail Blazers'
    ,'Clippers':'LakersClippers'
    ,'Lakers':'LakersClippers'
    ,'Knicks':'KnicksNets'
    ,'Nets':'KnicksNets'}


def new2(on):
    if on in dic2:
        res=dic2[on]
    else:
        res=on
    return res

def sec3(new):
    new=new.replace('*','')
    new=new.split('(')[0].strip()
    return new
nba_df['Full_Name']=nba_df['Full_Name'].apply(sec3)
nba_df['Full_Name']=nba_df['Full_Name'].apply(sec)
nba_df['Full_Name']=nba_df['Full_Name'].apply(new2)
nba_df.head()

,team,W,L,W/L%,GB,PS/G,PA/G,SRS,year,League,Full_Name
0,Toronto Raptors* (1),59,23,0.72,—,111.7,103.9,7.29,2018,NBA,Raptors
1,Boston Celtics* (2),55,27,0.6709999999999999,4.0,104.0,100.4,3.23,2018,NBA,Celtics
2,Philadelphia 76ers* (3),52,30,0.634,7.0,109.8,105.3,4.3,2018,NBA,76ers
3,Cleveland Cavaliers* (4),50,32,0.61,9.0,110.9,109.9,0.59,2018,NBA,Cavaliers
4,Indiana Pacers* (5),48,34,0.585,11.0,105.6,104.2,1.18,2018,NBA,Pacers


In [5]:
citiestwo=cities[['Metropolitan area','Population (2016 est.)[8]','NBA']]
citiestwo['NBA2']=citiestwo['NBA'].apply(sec2)
citiestwo.head()

,Metropolitan area,Population (2016 est.)[8],NBA,NBA2
0,New York City,20153634,KnicksNets,KnicksNets
1,Los Angeles,13310447,LakersClippers,LakersClippers
2,San Francisco Bay Area,6657982,Warriors,Warriors
3,Chicago,9512999,Bulls[note 9],Bulls
4,Dallas–Fort Worth,7233323,Mavericks,Mavericks


In [6]:
mergedtwo=citiestwo.merge( nba_df,left_on='NBA2' ,right_on='Full_Name')
mergedtwo=mergedtwo[['Metropolitan area','W/L%','Population (2016 est.)[8]','NBA2']]
mergedtwo['Population (2016 est.)[8]']= mergedtwo['Population (2016 est.)[8]'].apply(lambda x: float(x))
mergedtwo['W/L%']= mergedtwo['W/L%'].apply(lambda x: float(x))
mergedtwo=mergedtwo.groupby('Metropolitan area').agg({'W/L%':np.nanmean,'Population (2016 est.)[8]':np.nanmean})
mergedtwo.head()

,W/L%,Population (2016 est.)[8]
Metropolitan area,,
Atlanta,0.293,5789700.0
Boston,0.671,4794447.0
Charlotte,0.439,2474314.0
Chicago,0.329,9512999.0
Cleveland,0.610,2055612.0


In [7]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re



def nba_correlation():
    # YOUR CODE HERE
#    raise NotImplementedError()
    
    population_by_region = mergedtwo['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = mergedtwo['W/L%'] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

#    assert len(population_by_region)==len(win_loss_by_region), "Q2: Your lists must be the same length"
#    assert len(population_by_region)==30, "Q2: There should be 30 teams being analysed for NBA"

    return float(stats.pearsonr(population_by_region, win_loss_by_region)[0])

In [8]:
nba_correlation()

-0.17636350642182938

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [9]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]



In [10]:
mlb_df=mlb_df[mlb_df['year']==2018]
mlb_df['Full_Name3']=mlb_df['team']
def seco(name):
    try:
        name2=str(name.split()[1])+' '+str(name.split()[2])
    except:
        name2=str(name.split()[1])
    return name2
mlb_df['Full_Name3']=mlb_df['Full_Name3'].apply(seco)
mlb_df['Full_Name3'].head()

0         Red Sox
1    York Yankees
2        Bay Rays
3       Blue Jays
4         Orioles
Name: Full_Name3, dtype: object

In [11]:
citiesthree=cities[['Metropolitan area','Population (2016 est.)[8]','MLB']]
citiesthree['MLB2']=citiesthree['MLB'].apply(sec2)

dic3={'York Yankees':'YankeesMets'
    ,'Bay Rays':'Rays'
    ,'White Sox':'CubsWhite Sox'
    ,'City Royals':'Royals'
    ,'Athletics':'GiantsAthletics'
    ,'Angeles Angels':'DodgersAngels'
    ,'York Mets':'YankeesMets'
    ,'Cubs':'CubsWhite Sox'
    ,'Louis Cardinals':'Cardinals'
    ,'Angeles Dodgers':'DodgersAngels'
    ,'Francisco Giants':'GiantsAthletics'
    ,'Diego Padres':'Padres'}
def new3(on):
    if on in dic3:
        res=dic3[on]
    else:
        res=on
    return res
mlb_df['Full_Name3']=mlb_df['Full_Name3'].apply(new3)
mlb_df.head()

,team,W,L,W-L%,GB,year,League,Full_Name3
0,Boston Red Sox,108,54,0.667,--,2018,MLB,Red Sox
1,New York Yankees,100,62,0.617,8.0,2018,MLB,YankeesMets
2,Tampa Bay Rays,90,72,0.556,18.0,2018,MLB,Rays
3,Toronto Blue Jays,73,89,0.451,35.0,2018,MLB,Blue Jays
4,Baltimore Orioles,47,115,0.290,61.0,2018,MLB,Orioles


In [12]:
#citiesthree['MLB2'].unique

In [13]:
mergedthree=citiesthree.merge( mlb_df,left_on='MLB2' ,right_on='Full_Name3')
mergedthree=mergedthree[['Metropolitan area','W-L%','Population (2016 est.)[8]','MLB2']]
mergedthree['Population (2016 est.)[8]']= mergedthree['Population (2016 est.)[8]'].apply(lambda x: float(x))
mergedthree['W-L%']= mergedthree['W-L%'].apply(lambda x: float(x))
mergedthree=mergedthree.groupby('Metropolitan area').agg({'W-L%':np.nanmean,'Population (2016 est.)[8]':np.nanmean})
mergedthree.head()

,W-L%,Population (2016 est.)[8]
Metropolitan area,,
Atlanta,0.556,5789700.0
Baltimore,0.290,2798886.0
Boston,0.667,4794447.0
Chicago,0.483,9512999.0
Cincinnati,0.414,2165139.0


In [14]:
def mlb_correlation(): 
    # YOUR CODE HERE
#    raise NotImplementedError()
    
    population_by_region = mergedthree['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = mergedthree['W-L%'] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

#    assert len(population_by_region)==len(win_loss_by_region), "Q3: Your lists must be the same length"
#    assert len(population_by_region)==30, "Q3: There should be 30 teams being analysed for MLB"

    return float(stats.pearsonr(population_by_region, win_loss_by_region)[0])

In [15]:
mlb_correlation()

0.15003737475409495

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [16]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]



In [17]:
nfl_df=nfl_df[nfl_df['year']==2018]
nfl_df['Full_Name4']=nfl_df['team']

def seco4(name):
    name1=name.replace('*','')
    name1=name1.replace('+','')
    name1=name1.strip()
    try:
        name2=str(name1.split()[1])+' '+str(name1.split()[2])
    except:
        name2=str(name1.split()[1])
    return name2
nfl_df['Full_Name4']=nfl_df['Full_Name4'].apply(seco4)



In [18]:

citiesfour=cities[['Metropolitan area','Population (2016 est.)[8]','NFL']]
citiesfour['NFL']=citiesfour['NFL'].apply(sec2)
citiesfour.head()

,Metropolitan area,Population (2016 est.)[8],NFL
0,New York City,20153634,GiantsJets
1,Los Angeles,13310447,RamsChargers
2,San Francisco Bay Area,6657982,49ersRaiders
3,Chicago,9512999,Bears
4,Dallas–Fort Worth,7233323,Cowboys


In [19]:


dic4={'Bay Packers':'Packers'
    ,'City Chiefs':'Chiefs'
    ,'Angeles Chargers':'RamsChargers'
    ,'Angeles Rams':'RamsChargers'
    ,'England Patriots':'Patriots'
    ,'Orleans Saints':'Saints'
    ,'York Giants':'GiantsJets'
    ,'York Jets':'GiantsJets'
    ,'Raiders ':'49ersRaiders'
    ,'Francisco 49ers':'49ersRaiders'
    ,'Bay Buccaneers':'Buccaneers'}
def new4(on):
    if on in dic4:
        res=dic4[on]
    else:
        res=on
    return res
nfl_df['Full_Name4']=nfl_df['Full_Name4'].apply(new4)

mergedfour=citiesfour.merge( nfl_df,left_on='NFL' ,right_on='Full_Name4')
mergedfour=mergedfour[['Metropolitan area','W-L%','Population (2016 est.)[8]','NFL']]
mergedfour['Population (2016 est.)[8]']= mergedfour['Population (2016 est.)[8]'].apply(lambda x: float(x))
mergedfour['W-L%']= mergedfour['W-L%'].apply(lambda x: float(x))
mergedfour=mergedfour.groupby('Metropolitan area').agg({'W-L%':np.nanmean,'Population (2016 est.)[8]':np.nanmean})
len(mergedfour)



29

In [25]:
def nfl_correlation(): 
    # YOUR CODE HERE
#    raise NotImplementedError()
    
    population_by_region = mergedfour['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = mergedfour['W-L%'] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

#    assert len(population_by_region)==len(win_loss_by_region), "Q4: Your lists must be the same length"
#    assert len(population_by_region)==32, "Q4: There should be 32 teams being analysed for NHL"

    return float(stats.pearsonr(population_by_region, win_loss_by_region)[0])

In [26]:
nfl_correlation()

0.004282141436393017

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [123]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]



In [124]:
mergedone5=merged3[:]
mergedone5['NHL1']='NHL1'
mergedtwo5=mergedtwo[:]
mergedtwo5['NBA1']='NBA1'
mergedthree5=mergedthree[:]
mergedthree5['MLB1']='MLB1'
mergedfour5=mergedfour[:]
mergedfour5['NFL1']='NFL1'
mergedone5.head()

,WL,Population (2016 est.)[8],NHL1
Metropolitan area,,,
Boston,0.714286,4794447.0,NHL1
Buffalo,0.357143,1132804.0,NHL1
Calgary,0.513889,1392609.0,NHL1
Chicago,0.458333,9512999.0,NHL1
Columbus,0.600000,2041520.0,NHL1


In [125]:
dfr1=mergedone5.merge(mergedtwo5, on='Metropolitan area', how='outer')
dfr2=dfr1.merge(mergedthree5,on='Metropolitan area', how='outer')
dfr3=dfr2.merge(mergedfour5,on='Metropolitan area', how='outer')
dfr3.head()

,WL,Population (2016 est.)[8]_x,NHL1,W/L%,Population (2016 est.)[8]_y,NBA1,W-L%_x,Population (2016 est.)[8]_x,MLB1,W-L%_y,Population (2016 est.)[8]_y,NFL1
Metropolitan area,,,,,,,,,,,,
Boston,0.714286,4794447.0,NHL1,0.671,4794447.0,NBA1,0.667,4794447.0,MLB1,0.688,4794447.0,NFL1
Buffalo,0.357143,1132804.0,NHL1,NaN,NaN,NaN,NaN,NaN,NaN,0.375,1132804.0,NFL1
Calgary,0.513889,1392609.0,NHL1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Chicago,0.458333,9512999.0,NHL1,0.329,9512999.0,NBA1,0.483,9512999.0,MLB1,0.750,9512999.0,NFL1
Columbus,0.600000,2041520.0,NHL1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [135]:
#pair1_NHL_NBA
p1=dfr3[['WL','W/L%']].dropna(axis=0)
pval1=stats.ttest_rel(p1.iloc[:,0],p1.iloc[:,1])[1]

#pair2_NHL_MLB
p2=dfr3[['WL','W-L%_x']].dropna(axis=0)
pval2=stats.ttest_rel(p2.iloc[:,0],p2.iloc[:,1])[1]

#pair3_NHL_NFL
p3=dfr3[['WL','W-L%_y']].dropna(axis=0)
pval3=stats.ttest_rel(p3.iloc[:,0],p3.iloc[:,1])[1]

#pair4_NBA_MLB
p4=dfr3[['W/L%','W-L%_x']].dropna(axis=0)
pval4=stats.ttest_rel(p4.iloc[:,0],p4.iloc[:,1])[1]

#pair5_NBA_NFL
p5=dfr3[['W/L%','W-L%_y']].dropna(axis=0)
pval5=stats.ttest_rel(p5.iloc[:,0],p5.iloc[:,1])[1]

#pair6_MLB_NFL
p6=dfr3[['W-L%_x','W-L%_y']].dropna(axis=0)
pval6=stats.ttest_rel(p6.iloc[:,0],p6.iloc[:,1])[1]

(pval1,pval2,pval3,pval4,pval5,pval6)
lst=['NHL','NBA','MLB','NFL']
dfr4=dfr3.rename(mapper={'WL':'NHL','W/L%':'NBA','W-L%_x':'MLB','W-L%_y':'NFL'}, axis=1)
dfr5=dfr4[['NHL','NBA','MLB','NFL']]
#dfr5.groupby(level=0).agg(stats.ttest_rel)
sports = ['NFL', 'NBA', 'NHL', 'MLB']
p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
p_values['NBA']=[pval5,np.nan, pval1, pval4]
p_values['NHL']=[pval3,pval1,np.nan,pval2]
p_values['NFL']=[np.nan,pval5, pval3, pval6]
p_values['MLB']=[pval6,pval4,pval2,np.nan]

p_values

,NFL,NBA,NHL,MLB
NFL,NaN,0.937509,0.030318,0.803459
NBA,0.937509,NaN,0.022386,0.949566
NHL,0.030318,0.022386,NaN,0.000703
MLB,0.803459,0.949566,0.000703,NaN


In [138]:
def sports_team_performance():
    # YOUR CODE HERE
#    raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
#    sports = ['NFL', 'NBA', 'NHL', 'MLB']
#    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
#    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NFL p-value should be around 0.02"
#    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The NBA-NFL p-value should be around 0.80"
    return p_values

In [139]:
sports_team_performance()

,NFL,NBA,NHL,MLB
NFL,NaN,0.937509,0.030318,0.803459
NBA,0.937509,NaN,0.022386,0.949566
NHL,0.030318,0.022386,NaN,0.000703
MLB,0.803459,0.949566,0.000703,NaN
